In [65]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

In [66]:
df = pd.read_csv('country_codes.csv')
df2 = pd.read_csv('gdp_data.csv')

In [67]:
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   country_code  217 non-null    object
 1   region        217 non-null    object
 2   income_group  217 non-null    object
dtypes: object(3)
memory usage: 5.2+ KB


In [68]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13365 entries, 0 to 13364
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country_name  13365 non-null  object 
 1   country_code  13365 non-null  object 
 2   year          13365 non-null  int64  
 3   value         13365 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 417.8+ KB


In [69]:
np.random.seed(50) #gets the randomized seed

# Generate a random mask based on the random seed
random_mask = np.random.choice([True, False], size=len(df2))

# Use the mask to set random values in the "value" column to NaN
df2.loc[random_mask, 'value'] = np.nan

df2

,country_name,country_code,year,value
0,Afghanistan,AFG,1960,NaN
1,Afghanistan,AFG,1961,NaN
2,Afghanistan,AFG,1962,5.466667e+08
...,...,...,...,...
13362,Zimbabwe,ZWE,2020,NaN
13363,Zimbabwe,ZWE,2021,2.837124e+10
13364,Zimbabwe,ZWE,2022,2.067806e+10


In [70]:
df2_sum = df2.isnull().sum()
print(df2_sum)
df2['value'].fillna(df2['value'].mean(), inplace=True)
df2.head()

country_name       0
country_code       0
year               0
value           6641
dtype: int64


,country_name,country_code,year,value
0,Afghanistan,AFG,1960,1.193038e+12
1,Afghanistan,AFG,1961,1.193038e+12
2,Afghanistan,AFG,1962,5.466667e+08
3,Afghanistan,AFG,1963,7.511112e+08
4,Afghanistan,AFG,1964,8.000000e+08


In [71]:
df2.rename(columns=  {'year' : 'years' }, inplace= True)# Rename a column using df.rename()

df2

,country_name,country_code,years,value
0,Afghanistan,AFG,1960,1.193038e+12
1,Afghanistan,AFG,1961,1.193038e+12
2,Afghanistan,AFG,1962,5.466667e+08
...,...,...,...,...
13362,Zimbabwe,ZWE,2020,1.193038e+12
13363,Zimbabwe,ZWE,2021,2.837124e+10
13364,Zimbabwe,ZWE,2022,2.067806e+10


In [72]:
#df.columns =  ['country_codes', 'regions' ,'income_group' ] #directly modefy the name by using df.column

df

,country_code,region,income_group
0,ABW,Latin America & Caribbean,High income
1,AFG,South Asia,Low income
2,AGO,Sub-Saharan Africa,Lower middle income
...,...,...,...
214,ZAF,Sub-Saharan Africa,Upper middle income
215,ZMB,Sub-Saharan Africa,Lower middle income
216,ZWE,Sub-Saharan Africa,Lower middle income


In [73]:
df.columns =  ['country_code', 'regions' ,'income_group' ]

df

,country_code,regions,income_group
0,ABW,Latin America & Caribbean,High income
1,AFG,South Asia,Low income
2,AGO,Sub-Saharan Africa,Lower middle income
...,...,...,...
214,ZAF,Sub-Saharan Africa,Upper middle income
215,ZMB,Sub-Saharan Africa,Lower middle income
216,ZWE,Sub-Saharan Africa,Lower middle income


In [74]:
merged_df = df.merge(df2, on='country_code', how='outer')#df.merge combines the data 
                                                         #frame based on a common key
merged_df

,country_code,regions,income_group,country_name,years,value
0,ABW,Latin America & Caribbean,High income,Aruba,1986.0,1.193038e+12
1,ABW,Latin America & Caribbean,High income,Aruba,1987.0,1.193038e+12
2,ABW,Latin America & Caribbean,High income,Aruba,1988.0,1.193038e+12
...,...,...,...,...,...,...
13365,WLD,NaN,NaN,World,2020.0,8.520000e+13
13366,WLD,NaN,NaN,World,2021.0,1.193038e+12
13367,WLD,NaN,NaN,World,2022.0,1.193038e+12


In [96]:
merged_sum = merged_df.isnull().sum()
print(merged_sum)


country_code    0
country_name    3
years           3
value           0
dtype: int64


In [100]:
thresholds = len(merged_df) - 5#make a threshold 

merged_df.dropna(axis= 1, thresh=thresholds )#only drop the columns that meets the threshold

merged_df

,country_code,country_name,years,value
0,ABW,Aruba,1986.0,1.193038e+12
1,ABW,Aruba,1987.0,1.193038e+12
2,ABW,Aruba,1988.0,1.193038e+12
...,...,...,...,...
13365,WLD,World,2020.0,8.520000e+13
13366,WLD,World,2021.0,1.193038e+12
13367,WLD,World,2022.0,1.193038e+12


In [101]:
max_value = merged_df['value'].agg('max')
print(f"Maximum value: {max_value}")
min_value = merged_df['value'].agg('min')
print(f"Minimum value: {min_value}")

Maximum value: 87700000000000.0
Minimum value: 9122751.45


In [102]:
# Define a custom function to format the values (removes the 'e+')
def format_value(value):
    return f'{value:.2f}'  # Adjust the precision as needed 

# Apply the custom function to the 'value' column using apply
merged_df['value'] = merged_df['value'].apply(format_value)



In [103]:
imputer = SimpleImputer(strategy='mean')#uses simpleimputer using the strategy of 
                                        #changing missing value by changing it using the mean of the column

imputer.fit(merged_df[['value']])#uses .fit to ensure that only the column 'value' will be changed

merged_df['value'] = imputer.transform(merged_df[['value']]) # transform the 'value' column in the original DataFrame df to replace the missing values with the calculated mean

merged_df

,country_code,country_name,years,value
0,ABW,Aruba,1986.0,1.193038e+12
1,ABW,Aruba,1987.0,1.193038e+12
2,ABW,Aruba,1988.0,1.193038e+12
...,...,...,...,...
13365,WLD,World,2020.0,8.520000e+13
13366,WLD,World,2021.0,1.193038e+12
13367,WLD,World,2022.0,1.193038e+12


In [80]:
pd.set_option("display.max_rows", 7) # set the maximum number of 
                                        # rows shown to nonne(everything) can be changed to any number

df

,country_code,regions,income_group
0,ABW,Latin America & Caribbean,High income
1,AFG,South Asia,Low income
2,AGO,Sub-Saharan Africa,Lower middle income
...,...,...,...
214,ZAF,Sub-Saharan Africa,Upper middle income
215,ZMB,Sub-Saharan Africa,Lower middle income
216,ZWE,Sub-Saharan Africa,Lower middle income


In [81]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13368 entries, 0 to 13367
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country_code  13368 non-null  object 
 1   regions       10551 non-null  object 
 2   income_group  10551 non-null  object 
 3   country_name  13365 non-null  object 
 4   years         13365 non-null  float64
 5   value         13368 non-null  float64
dtypes: float64(2), object(4)
memory usage: 626.8+ KB


In [82]:
non_null_counts = merged_df.count()

# Define the threshold (e.g., 13367)
threshold = 13367

# Use boolean indexing to filter columns based on the number of non-null values
selected_columns = non_null_counts[non_null_counts > threshold].index

# Display only the selected columns
result_df = merged_df[selected_columns]

print(result_df)

      country_code         value
0              ABW  1.193038e+12
1              ABW  1.193038e+12
2              ABW  1.193038e+12
...            ...           ...
13365          WLD  8.520000e+13
13366          WLD  1.193038e+12
13367          WLD  1.193038e+12

[13368 rows x 2 columns]


In [83]:
df.melt() #converts dataframe from wide format into a long format



,variable,value
0,country_code,ABW
1,country_code,AFG
2,country_code,AGO
...,...,...
648,income_group,Upper middle income
649,income_group,Lower middle income
650,income_group,Lower middle income


In [84]:
# Check the column names in your DataFrame
print(df.columns)

df.pivot(index='country_codes', columns='regions', values='income_group')#opposite of df.melt()

Index(['country_code', 'regions', 'income_group'], dtype='object')


KeyError: 'country_codes'

,country_codes,regions,income_group
0,ABW,Latin America & Caribbean,High income
1,AFG,South Asia,Low income
...,...,...,...
215,ZMB,Sub-Saharan Africa,Lower middle income
216,ZWE,Sub-Saharan Africa,Lower middle income
